## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Honningsvag,70.9821,25.9704,31.95,54,69,3.94,NO,2021-05-02 22:05:09,snow
1,1,Ushuaia,-54.8000,-68.3000,35.60,75,75,27.63,AR,2021-05-02 22:07:20,snow
2,2,Rikitea,-23.1203,-134.9692,77.25,78,66,15.50,PF,2021-05-02 22:07:54,light rain
3,3,Hilo,19.7297,-155.0900,77.00,73,90,6.91,US,2021-05-02 22:00:05,light rain
4,4,Ixtapa,20.7000,-105.2000,80.60,61,1,17.27,MX,2021-05-02 22:07:38,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
2,2,Rikitea,-23.1203,-134.9692,77.25,78,66,15.50,PF,2021-05-02 22:07:54,light rain
3,3,Hilo,19.7297,-155.0900,77.00,73,90,6.91,US,2021-05-02 22:00:05,light rain
4,4,Ixtapa,20.7000,-105.2000,80.60,61,1,17.27,MX,2021-05-02 22:07:38,clear sky
7,7,Butaritari,3.0707,172.7902,82.63,75,100,14.32,KI,2021-05-02 21:59:51,light rain
9,9,Cabo San Lucas,22.8909,-109.9124,87.01,26,1,18.41,MX,2021-05-02 21:59:21,clear sky
11,11,Palhi,10.6498,124.8062,79.74,83,33,9.10,PH,2021-05-02 22:08:03,scattered clouds
25,25,Pailon,-17.6500,-62.7500,84.20,58,40,11.50,BO,2021-05-02 22:08:20,scattered clouds
27,27,Victoria,22.2855,114.1577,77.00,88,99,19.48,HK,2021-05-02 21:59:30,light rain
29,29,Saint Marys,30.7305,-81.5465,78.01,100,1,8.05,US,2021-05-02 22:08:24,clear sky
31,31,Hamilton,39.1834,-84.5333,75.20,46,1,12.66,US,2021-05-02 21:58:17,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                214
City                   214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Country                214
Date                   214
Current Description    214
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= clean_df.dropna()
#clean_df.count()

City_ID                214
City                   214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Country                214
Date                   214
Current Description    214
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,77.25,light rain,-23.1203,-134.9692,
3,Hilo,US,77.00,light rain,19.7297,-155.0900,
4,Ixtapa,MX,80.60,clear sky,20.7000,-105.2000,
7,Butaritari,KI,82.63,light rain,3.0707,172.7902,
9,Cabo San Lucas,MX,87.01,clear sky,22.8909,-109.9124,
11,Palhi,PH,79.74,scattered clouds,10.6498,124.8062,
25,Pailon,BO,84.20,scattered clouds,-17.6500,-62.7500,
27,Victoria,HK,77.00,light rain,22.2855,114.1577,
29,Saint Marys,US,78.01,clear sky,30.7305,-81.5465,
31,Hamilton,US,75.20,clear sky,39.1834,-84.5333,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
# hotel_df.count()
clean_hotel_df= hotel_df.dropna()
clean_hotel_df.count()

City                   214
Country                214
Max Temp               214
Current Description    214
Lat                    214
Lng                    214
Hotel Name             214
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    


In [30]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations,weight=max_temp, dissipating=False,
#            max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))